In [99]:
import numpy as np
import librosa
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [103]:
class VoiceprintExtractor:
    def __init__(self, sample_rate=16000, duration=3):
        self.sample_rate = sample_rate
        self.duration = duration
        self.model = self._build_feature_extractor()
        
    def _build_feature_extractor(self):
        inputs = Input(shape=(128, 40))
        x = LSTM(64, return_sequences=True)(inputs)
        x = Dropout(0.3)(x)
        x = LSTM(32)(x)
        x = Dense(64, activation='relu')(x)
        outputs = Dense(32, activation=None)(x)
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer='adam', loss='mse')
        return model
    
    def extract_features(self, audio_data):
        try:
            mfccs = librosa.feature.mfcc(y=audio_data, 
                                        sr=self.sample_rate,
                                        n_mfcc=40)
            mfccs = mfccs.T
            if mfccs.shape[0] < 128:
                mfccs = np.pad(mfccs, ((0, 128 - mfccs.shape[0]), (0, 0)))
            else:
                mfccs = mfccs[:128, :]
            return mfccs
        except Exception as e:
            print(f"Error extracting features: {e}")
            return None
    
    def get_voiceprint(self, audio_data):
        features = self.extract_features(audio_data)
        if features is None:
            return None
        features = np.expand_dims(features, axis=0)
        embedding = self.model.predict(features, verbose=0)
        return embedding.flatten()

In [105]:
class ContextAwareness:
    def __init__(self):
        self.fuzzy_system = self._build_fuzzy_system()
        
    def _build_fuzzy_system(self):
        noise_level = ctrl.Antecedent(np.arange(0, 101, 1), 'noise_level')
        movement = ctrl.Antecedent(np.arange(0, 101, 1), 'movement')
        sensitivity = ctrl.Consequent(np.arange(0, 101, 1), 'sensitivity')
        power_mode = ctrl.Consequent(np.arange(0, 101, 1), 'power_mode')
        
        noise_level['low'] = fuzz.trimf(noise_level.universe, [0, 0, 40])
        noise_level['medium'] = fuzz.trimf(noise_level.universe, [20, 50, 80])
        noise_level['high'] = fuzz.trimf(noise_level.universe, [60, 100, 100])
        
        movement['stationary'] = fuzz.trimf(movement.universe, [0, 0, 30])
        movement['walking'] = fuzz.trimf(movement.universe, [20, 50, 80])
        movement['running'] = fuzz.trimf(movement.universe, [70, 100, 100])
    
        sensitivity['low'] = fuzz.trimf(sensitivity.universe, [0, 0, 40])
        sensitivity['medium'] = fuzz.trimf(sensitivity.universe, [30, 50, 70])
        sensitivity['high'] = fuzz.trimf(sensitivity.universe, [60, 100, 100])
        
        power_mode['eco'] = fuzz.trimf(power_mode.universe, [0, 0, 40])
        power_mode['balanced'] = fuzz.trimf(power_mode.universe, [30, 50, 70])
        power_mode['performance'] = fuzz.trimf(power_mode.universe, [60, 100, 100])
    
        rule1 = ctrl.Rule(noise_level['high'], sensitivity['high'])
        rule2 = ctrl.Rule(noise_level['low'], sensitivity['low'])
        rule3 = ctrl.Rule(movement['running'], power_mode['performance'])
        rule4 = ctrl.Rule(movement['stationary'], power_mode['eco'])
        rule5 = ctrl.Rule(movement['walking'] & noise_level['medium'], 
                         (sensitivity['medium'], power_mode['balanced']))

        system = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])
        return ctrl.ControlSystemSimulation(system)
    
    def detect_noise_level(self, audio_data):
        if len(audio_data) > 0:
            energy = np.sqrt(np.mean(np.square(audio_data)))
            noise_level = min(100, max(0, energy * 100))
            return noise_level
        return 0
    
    def detect_movement(self, acceleration_data=None):
        if acceleration_data is None:
            return np.random.uniform(0, 100)
        else:
            movement = np.mean(np.abs(acceleration_data))
            return min(100, max(0, movement * 10))
            
    def adapt_to_context(self, noise_level, movement):
        try:
            self.fuzzy_system.input['noise_level'] = noise_level
            self.fuzzy_system.input['movement'] = movement
    
            self.fuzzy_system.compute()
            sensitivity = self.fuzzy_system.output['sensitivity']
            power_mode = self.fuzzy_system.output['power_mode']
            return {
                'sensitivity': sensitivity,
                'power_mode': power_mode
            }
        except Exception as e:
            print(f"Error in fuzzy adaptation: {e}")
            return {
                'sensitivity': 50, 
                'power_mode': 50
            }

In [107]:
class EnergyOptimizer:
    def __init__(self):
        self.power_states = {
            'sleep': 5,      
            'listening': 14, 
            'processing': 25
        }
        self.current_state = 'sleep'
        self.current_power = self.power_states['sleep']
        
    def optimize_power(self, context_data):
        power_mode = context_data.get('power_mode', 50)
        if power_mode < 30:
            self.set_power_state('sleep')
        elif power_mode < 70:
            self.set_power_state('listening')
        else:
            self.set_power_state('processing')
        return self.current_power
        
    def set_power_state(self, state):
        if state in self.power_states:
            self.current_state = state
            self.current_power = self.power_states[state]
            print(f"Power state changed to {state} ({self.current_power} mW)")
            return True
        return False
        
    def estimate_battery_life(self, battery_capacity_mAh=1000, voltage=3.7):
        battery_capacity_mWh = battery_capacity_mAh * voltage
        estimated_hours = battery_capacity_mWh / self.current_power
        return estimated_hours

In [109]:
class EcoContextVoxID:
    def __init__(self):
        self.voiceprint_extractor = VoiceprintExtractor()
        self.context_awareness = ContextAwareness()
        self.energy_optimizer = EnergyOptimizer()
        self.users = {} 
        self.current_user = None
        
    def register_user(self, user_id, audio_data):
        voiceprint = self.voiceprint_extractor.get_voiceprint(audio_data)
        if voiceprint is not None:
            self.users[user_id] = voiceprint
            print(f"User {user_id} registered successfully")
            return True
        print(f"Failed to register user {user_id}")
        return False
        
    def identify_user(self, audio_data):
        if not self.users:
            print("No users registered")
            return None
            
        test_voiceprint = self.voiceprint_extractor.get_voiceprint(audio_data)
        if test_voiceprint is None:
            return None
        best_match = None
        best_score = -1
        
        for user_id, voiceprint in self.users.items():
            similarity = self._cosine_similarity(test_voiceprint, voiceprint)
            if similarity > best_score:
                best_score = similarity
                best_match = user_id
        
        if best_score > 0.85:
            self.current_user = best_match
            return best_match
        return None
        
    def _cosine_similarity(self, a, b):
        return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
        
    def process_context(self, audio_data, acceleration_data=None):
        noise_level = self.context_awareness.detect_noise_level(audio_data)
        movement = self.context_awareness.detect_movement(acceleration_data)
        context_data = self.context_awareness.adapt_to_context(noise_level, movement)
        self.energy_optimizer.optimize_power(context_data)
        return context_data

In [ ]:
def generate_fake_audio(duration=3, sample_rate=16000, noise_level=0.1):
    return np.random.normal(0, noise_level, int(duration * sample_rate))

vox_system = EcoContextVoxID()

print("Creating simulated audio data for users...")
user1_samples = [generate_fake_audio(noise_level=0.1) for _ in range(3)]
user2_samples = [generate_fake_audio(noise_level=0.15) for _ in range(3)]

# %%
# Register users
print("\nRegistering users...")
vox_system.register_user("Alice", user1_samples[0])
vox_system.register_user("Bob", user2_samples[0])

# %%
# Test user identification
print("\nTesting user identification...")
test_audio = user1_samples[1]  # Use a different sample from the same user
identified_user = vox_system.identify_user(test_audio)
print(f"Identified user: {identified_user}")

# %%
# Demonstrate context adaptation
print("\nDemonstrating context adaptation...")
test_contexts = [
    {"name": "Quiet home", "noise": 0.05, "movement": 5},
    {"name": "Noisy street", "noise": 0.5, "movement": 40},
    {"name": "Running outdoors", "noise": 0.2, "movement": 90}
]

plt.figure(figsize=(10, 6))
power_values = []
sensitivity_values = []

for i, context in enumerate(test_contexts):
    print(f"\nSimulating context: {context['name']}")
    # Generate fake audio with appropriate noise level
    audio = generate_fake_audio(noise_level=context["noise"])
    
    # Process the context
    context_result = vox_system.process_context(
        audio,
        np.array([context["movement"]/10])
    )
    
    print(f"Sensitivity level: {context_result['sensitivity']:.1f}/100")
    print(f"Power mode: {context_result['power_mode']:.1f}/100")
    print(f"Current power usage: {vox_system.energy_optimizer.current_power} mW")
    
    # Simulate battery life
    battery_life = vox_system.energy_optimizer.estimate_battery_life()
    print(f"Estimated battery life: {battery_life:.1f} hours")
    
    power_values.append(vox_system.energy_optimizer.current_power)
    sensitivity_values.append(context_result['sensitivity'])

# %%
# Plot results
contexts = [c["name"] for c in test_contexts]
    
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.bar(contexts, power_values)
plt.title('Power Consumption by Context')
plt.ylabel('Power (mW)')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
plt.bar(contexts, sensitivity_values)
plt.title('Sensitivity Level by Context')
plt.ylabel('Sensitivity Level')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print("\nSystem demo completed!")